In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os, sys, getpass
user = getpass.getuser()
sys.path.append(f'/home/{user}/codes/hpe_library/')
from lib_import import *
from my_utils import *
os.chdir('/home/hrai/codes/MotionBERT')

In [2]:
h36m_3d_world, h36m_cam_param = load_h36m()

subject = 'S1'
action = 'Walking'
pose3d_s1_walking = h36m_3d_world[subject][action]['positions'] # 3d skeleton sequence wrt world CS
cam_info_s1_walking = h36m_3d_world[subject][action]['cameras']
cam_param_s1_walking = get_cam_param(cam_info_s1_walking, subject, h36m_cam_param)

# camera parameters
W, H = cam_param_s1_walking['54138969']['W'], cam_param_s1_walking['54138969']['H']
cam_54138969_ext = cam_param_s1_walking['54138969']['ext']
cam_54138969_int = cam_param_s1_walking['54138969']['int']
cam_54138969_proj = cam_param_s1_walking['54138969']['proj']
cam_54138969_origin = cam_param_s1_walking['54138969']['C']

==> Loading 3D data wrt World CS...


In [3]:
pose3d_s1_walking.shape

(3134, 17, 3)

In [27]:
import torch.nn.functional as F

def get_limb_lens(x):
    '''
        Input: (N, T, 17, 3)
        Output: (N, T, 16)
    '''
    limbs_id = [[0,1], [1,2], [2,3],
         [0,4], [4,5], [5,6],
         [0,7], [7,8], [8,9], [9,10],
         [8,11], [11,12], [12,13],
         [8,14], [14,15], [15,16]
        ]
    limbs = x[:,:,limbs_id,:]
    print(limbs.shape)
    limbs = limbs[:,:,:,0,:]-limbs[:,:,:,1,:]
    limb_lens = torch.norm(limbs, dim=-1)
    return limb_lens

def get_angles(x):
    '''
        Input: (N, T, 17, 3)
        Output: (N, T, 16)
    '''
    limbs_id = [[0,1], [1,2], [2,3],
         [0,4], [4,5], [5,6],
         [0,7], [7,8], [8,9], [9,10],
         [8,11], [11,12], [12,13],
         [8,14], [14,15], [15,16]
        ]
    angle_id = [[ 0,  3],
                [ 0,  6],
                [ 3,  6],
                [ 0,  1],
                [ 1,  2],
                [ 3,  4],
                [ 4,  5],
                [ 6,  7],
                [ 7, 10],
                [ 7, 13],
                [ 8, 13],
                [10, 13],
                [ 7,  8],
                [ 8,  9],
                [10, 11],
                [11, 12],
                [13, 14],
                [14, 15] ]
    eps = 1e-7
    limbs = x[:,:,limbs_id,:]
    limbs = limbs[:,:,:,0,:]-limbs[:,:,:,1,:] # make limb vectors
    angles = limbs[:,:,angle_id,:] # select 2 vectors for each angle
    angle_cos = F.cosine_similarity(angles[:,:,:,0,:], angles[:,:,:,1,:], dim=-1)
    return torch.acos(angle_cos.clamp(-1+eps, 1-eps)) 

In [29]:
x = torch.tensor(pose3d_s1_walking).reshape(1, -1, 17, 3)
limb_lens = get_limb_lens(x).cpu().numpy()[0]
angles = get_angles(x).cpu().numpy()[0]
limb_lens.shape, angles.shape

torch.Size([1, 3134, 16, 2, 3])


((3134, 16), (3134, 18))

In [18]:
limb_lens.mean(axis=0), limb_lens.std(axis=0)

(array([0.13294382, 0.44288164, 0.4542153 , 0.13294384, 0.44288164,
        0.45421535, 0.23347603, 0.25707984, 0.12113259, 0.11500042,
        0.15103191, 0.2788739 , 0.2517276 , 0.1510315 , 0.2788864 ,
        0.25172478], dtype=float32),
 array([4.7681438e-06, 1.2969689e-05, 8.8397410e-06, 4.9860951e-06,
        1.2770107e-05, 8.7547105e-06, 2.2893942e-06, 2.1611881e-06,
        2.3475952e-06, 1.8082617e-06, 2.3134999e-06, 8.8824117e-06,
        5.8389469e-06, 8.0186034e-08, 6.5149479e-06, 3.8949820e-06],
       dtype=float32))

In [30]:
angles[0]

array([3.1411045 , 1.4650826 , 1.67651   , 1.697313  , 0.03596941,
       1.5486748 , 0.0914357 , 0.11426465, 2.1095095 , 2.123755  ,
       1.9761837 , 2.0478528 , 0.5529261 , 1.5220519 , 0.73272836,
       0.2104467 , 0.7446696 , 0.2675639 ], dtype=float32)